## 卷积神经网络
## tf.nn.conv2d
[TF-卷积函数 tf.nn.conv2d 介绍](https://www.cnblogs.com/qggg/p/6832342.html)
````python
tf.nn.conv2d(input, # [batch, in_height, in_width, in_channels]
            filter, # [filter_height, filter_width, in_channels, out_channels]
            strides, # [1,1,1,1]
            padding, # SAME VALID
            use_cudnn_on_gpu=True,
            data_format='NHWC',
            dilations=[1, 1, 1, 1],
            name=None)
````
- input: [batch, in_height, in_width, in_channels] 4-D的 Tensor (float32/float64)
       batch_size,高度，宽度，通道数
- filter：[filter_height, filter_width, in_channels, out_channels]
        卷积核高度，   卷积核宽度      输入通道数    输出通道数(卷积核个数)
- strides：步长 [1,1,1,1] 1-D向量，长度为4
- padding：填充 SAME VALID
- use_cudnn_on_gpu：是否GPU加速

结果返回一个Tensor，这个输出，就是我们常说的 feature map，shape仍然是 [batch, height, width, channels] 这种形式。

### 图像卷积
#### 3x3图像，1通道，1x1卷积核
1.考虑一种最简单的情况，现在有一张3×3单通道的图像（对应的shape：[1，3，3，1]），用一个1×1的卷积核（对应的shape：[1，1，1，1]）去做卷积，最后会得到一张3×3的feature map

In [4]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,3,3,1]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([1,1,1,1]))
op1 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op1),'\n')
    print(sess.run(op1).shape)

[[[[-0.06386475]
   [ 0.30251193]
   [-0.36254457]]

  [[-0.1863834 ]
   [-0.11046342]
   [ 0.12128225]]

  [[-0.16598591]
   [-0.06247617]
   [ 0.10344568]]]] 

(1, 3, 3, 1)


#### 3x3图像，5通道，1x1卷积核
2.增加图片的通道数，使用一张3×3五通道的图像（对应的shape：[1，3，3，5]），用一个1×1的卷积核（对应的shape：[1，1，1，1]）去做卷积，仍然是一张3×3的feature map，这就相当于每一个像素点，卷积核都与该像素点的每一个通道做卷积。

In [5]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,3,3,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([1,1,5,1]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[ 2.611188 ]
   [ 1.1356436]
   [ 0.1728566]]

  [[ 1.7238054]
   [-2.65044  ]
   [-0.6026321]]

  [[-0.9376406]
   [-2.1341398]
   [ 0.5801886]]]] 

(1, 3, 3, 1)


#### 3x3图像，5通道，3x3卷积核，文本卷积维度和图像一样大，所以卷积之后只有1列
3.把卷积核扩大，现在用3×3的卷积核做卷积，最后的输出是一个值，相当于情况2的feature map所有像素点的值求和

In [6]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,3,3,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,3,5,1]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[-3.8528938]]]] 

(1, 1, 1, 1)


#### 5x5图像，5通道，3x3卷积核
4.使用更大的图片将情况2的图片扩大到5×5，仍然是3×3的卷积核，令步长为1，输出3×3的feature map

In [7]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,5,5,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,3,5,1]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[-3.3537188]
   [ 2.6631894]
   [10.7735815]]

  [[ 6.7866626]
   [-5.753437 ]
   [ 6.8379397]]

  [[-7.2338777]
   [-3.8412943]
   [11.663807 ]]]] 

(1, 3, 3, 1)


#### 5x5图像，5通道，3x3卷积核
5.上面我们一直令参数padding的值为‘VALID’，当其为‘SAME’时，表示卷积核可以停留在图像边缘，如下，输出5×5的feature map

In [8]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,5,5,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,3,5,1]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[ -4.1736193 ]
   [  7.2922435 ]
   [  2.9188058 ]
   [  0.49713266]
   [ -2.956664  ]]

  [[ -3.305164  ]
   [ -7.311406  ]
   [ -5.045771  ]
   [ -2.5354984 ]
   [ -7.40237   ]]

  [[  9.273168  ]
   [  1.2130424 ]
   [ -8.63011   ]
   [  8.675023  ]
   [  4.0911283 ]]

  [[ -2.295607  ]
   [  5.2230077 ]
   [-10.142306  ]
   [ -6.135029  ]
   [  1.3315554 ]]

  [[-11.159186  ]
   [ -3.5029335 ]
   [ -1.638276  ]
   [ -4.381499  ]
   [ -1.0199151 ]]]] 

(1, 5, 5, 1)


#### 5x5图像，5通道，3x3卷积核，3个卷积核
6.如果卷积核有多个

此时输出3张5×5的feature map

In [10]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,5,5,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,3,5,3]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[  2.7570508   -3.232238    -2.6215773 ]
   [  2.532285     1.9889098    3.87929   ]
   [ -3.187311     8.91769      3.224719  ]
   [  0.44387245  -9.403946     2.468867  ]
   [  0.21311586  -1.590601     5.749056  ]]

  [[ -2.6277003    6.488189     9.992645  ]
   [  1.5766766  -11.48576      0.6145782 ]
   [ -5.0482545   -0.96584886   4.0381684 ]
   [ -0.797274     2.4302173   -3.8855307 ]
   [ -2.6238062    2.05465      2.9259453 ]]

  [[  4.714437    -0.9536078   -2.9879472 ]
   [  1.5400691    1.5240853   -6.90153   ]
   [ -3.6736727    3.85059     -0.5918405 ]
   [  7.023252     2.9593654  -13.595696  ]
   [ -5.041815    -2.7133517    0.6385279 ]]

  [[  1.477376     0.47209492   5.653083  ]
   [ -0.39575818  14.780628    -1.5949147 ]
   [  2.378466   -11.533363    -0.4041656 ]
   [ -0.4129743    6.5807753   -2.7889323 ]
   [  6.1631317   -0.49479347   1.52246   ]]

  [[ -6.97586      1.1432166   -5.064254  ]
   [ -9.823753    -3.1042528   -1.6604922 ]
   [  9.015108   -10.424

#### 5x5图像，5通道，3x3卷积核，3个卷积核，步长为2
7.步长不为1的情况，文档里说了对于图片，因为只有两维，通常strides取[1，stride，stride，1]

In [11]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([1,5,5,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,3,5,3]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 2, 2, 1], padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[ 2.783487  -2.09441    4.733526 ]
   [-7.3059773 -3.108855   4.022243 ]
   [-4.050215   3.0158758 -4.1893964]]

  [[-6.3690815 -5.2265515  1.1703218]
   [ 9.0784235 -3.5745146 14.855592 ]
   [-0.4078823  4.0576644  4.617129 ]]

  [[ 3.339266  -5.4302483 -3.154387 ]
   [ 1.3765206  2.6518223  5.6584387]
   [-3.9308991  1.4282804 -3.4455342]]]] 

(1, 3, 3, 3)


#### 5x5图像，5通道，3x3卷积核，3个卷积核，步长为2，10张图像
8.如果batch值不为1，同时输入10张图

每张图，都有3张3×3的feature map，输出的shape就是[10，3，3，3]

In [12]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([10,5,5,5]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,3,5,3]))
op2 = tf.nn.conv2d(input, filter, strides=[1, 2, 2, 1], padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op2),'\n')
    print(sess.run(op2).shape)

[[[[  3.7294517   -3.107039    -3.3261778 ]
   [  5.441018    -4.0686336    5.619485  ]
   [  2.1390946    3.5674727   -1.5291997 ]]

  [[ -5.6689725   16.974398     2.381929  ]
   [  2.4008992   -0.6694184    1.378117  ]
   [  1.2934582    7.2192235    0.48349503]]

  [[  2.9121394   -0.4573097   -9.765212  ]
   [  1.0088806   -0.7046843    6.591536  ]
   [ -0.72504395   0.43721557  -4.999654  ]]]


 [[[  1.967625    -1.6568589   -6.145099  ]
   [ -4.151078   -10.529405    -2.047928  ]
   [  3.4548922    2.7491624    2.9001775 ]]

  [[ -1.6896939    3.1873543    6.188783  ]
   [ 11.703161     1.6971766   -4.8438787 ]
   [  0.9549799   -1.1131762    4.593415  ]]

  [[ -1.1170579    0.4810401   -1.3526723 ]
   [  2.529728    -1.1482326    3.7958796 ]
   [ -0.24976896   3.3091352   -6.729189  ]]]


 [[[ -3.33531     -1.9344107    2.8165019 ]
   [ -1.6785766   -2.8081656    7.2197647 ]
   [  1.7976431    2.8334517   -0.08083367]]

  [[  2.5362453   -0.68693405   2.2952533 ]
   [  1.523693

### 文本卷积
#### 5x5文本，1通道，2-gram卷积核，3个卷积核，10句话

In [17]:
import tensorflow as tf
#                 [batch, in_height, in_width, in_channels]
input = tf.Variable(tf.random_normal([10,5,5,1]))
#                 [filter_height, filter_width, in_channels, out_channels]
filter = tf.Variable(tf.random_normal([3,5,1,3]))
op1 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(op1),'\n')
    print(sess.run(op1).shape)

[[[[ -0.28560075   3.4164069    0.64287925]]

  [[  4.922906     6.569628    -4.377281  ]]

  [[  2.0096815   -0.9498653   -5.302     ]]]


 [[[ -0.30866227   0.65657634   0.08617933]]

  [[  2.1648352    2.0540233   -6.2501183 ]]

  [[  1.8437229   -3.3579445    0.648278  ]]]


 [[[ -6.248692    -8.374758     3.1102016 ]]

  [[ -4.6158714    1.0821313    2.8032086 ]]

  [[  1.912092     0.933113    -3.0924444 ]]]


 [[[  7.5399313    9.936766    -1.6083889 ]]

  [[  2.4991071   -0.3938322    5.2363515 ]]

  [[ -3.8184917    1.5327872   -0.9156568 ]]]


 [[[ -1.0705962    1.3645588   -2.2302496 ]]

  [[  0.9711383   -2.6879628   -2.1285567 ]]

  [[ -5.15031     -1.7857913   -0.64766765]]]


 [[[ -4.300858    -0.74519587   4.707138  ]]

  [[  1.1525508   -1.9355469    1.1351813 ]]

  [[ -1.930467     5.30831     -0.11006889]]]


 [[[ -0.30049637  -3.3917482   -0.98812234]]

  [[ -0.78466344  -3.508609     1.8363969 ]]

  [[  1.6145957    0.15216915  -0.27968606]]]


 [[[  1.201814     2

## tf.nn.max_pool
[TF-池化函数 tf.nn.max_pool 的介绍](https://www.cnblogs.com/qggg/p/6832705.html)

````python
tf.nn.max_pool(value, # [batch, in_height, in_width, in_channels]
                ksize, #  [1,in_height,in_width,1]
                strides, # [1,height,width,1]
                padding, # SAME VALID
                data_format='NHWC',
                name=None,)
````

- value: 池化层的输入，一般池化层在卷积层后面，输入通常是feature_map
        依然是 [batch, in_height, in_width, in_channels]
- ksize：池化窗口大小，4-D向量，一般是 [1,in_height,in_width,1]
         因为我们不想在 batch，in_channels上做池化，所以维度为1
- strides：步长 [1,height,width,1] 1-D向量，长度为4
- padding：填充 SAME VALID

返回一个Tensor，类型不变，shape仍然是 [batch, in_height, in_width, in_channels] 这种形式

#### 4x4图像，2通道，池化核2x2，步长1

卷积由 
[batch, in_height, in_width, in_channels]

[1,4,4,2]  ->  [1,3,3,2]

In [22]:
# 2x4x4
a=tf.constant([  
        [[1.0,2.0,3.0,4.0],  
        [5.0,6.0,7.0,8.0],  
        [8.0,7.0,6.0,5.0],  
        [4.0,3.0,2.0,1.0]], # 通道1图像
    
        [[4.0,3.0,2.0,1.0],  # 通道2图像
         [8.0,7.0,6.0,5.0],  
         [1.0,2.0,3.0,4.0],  
         [5.0,6.0,7.0,8.0]]  
    ])  

b=tf.reshape(a,[1,4,4,2])

max_pool_2x2 = tf.nn.max_pool(b,[1,2,2,1],[1,1,1,1],padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('池化前原始，image：\n')
    print(sess.run(a),'\n')
    print(sess.run(a).shape,'\n')
    print('池化前，reshape，image：\n')
    print(sess.run(b),'\n')
    print(sess.run(b).shape,'\n')
    print('池化后image：\n')
    print(sess.run(max_pool_2x2),'\n')
    print(sess.run(max_pool_2x2).shape,'\n')

池化前原始，image：

[[[1. 2. 3. 4.]
  [5. 6. 7. 8.]
  [8. 7. 6. 5.]
  [4. 3. 2. 1.]]

 [[4. 3. 2. 1.]
  [8. 7. 6. 5.]
  [1. 2. 3. 4.]
  [5. 6. 7. 8.]]] 

(2, 4, 4) 

池化前，reshape，image：

[[[[1. 2.]
   [3. 4.]
   [5. 6.]
   [7. 8.]]

  [[8. 7.]
   [6. 5.]
   [4. 3.]
   [2. 1.]]

  [[4. 3.]
   [2. 1.]
   [8. 7.]
   [6. 5.]]

  [[1. 2.]
   [3. 4.]
   [5. 6.]
   [7. 8.]]]] 

(1, 4, 4, 2) 

池化后image：

[[[[8. 7.]
   [6. 6.]
   [7. 8.]]

  [[8. 7.]
   [8. 7.]
   [8. 7.]]

  [[4. 4.]
   [8. 7.]
   [8. 8.]]]] 

(1, 3, 3, 2) 



#### 4x4图像，2通道，池化核2x2，步长2

卷积由 
[batch, in_height, in_width, in_channels]

[1,4,4,2]  ->  [1,2,2,2]

In [24]:
# 2x4x4
a=tf.constant([  
        [[1.0,2.0,3.0,4.0],  
        [5.0,6.0,7.0,8.0],  
        [8.0,7.0,6.0,5.0],  
        [4.0,3.0,2.0,1.0]], # 通道1图像
    
        [[4.0,3.0,2.0,1.0],  # 通道2图像
         [8.0,7.0,6.0,5.0],  
         [1.0,2.0,3.0,4.0],  
         [5.0,6.0,7.0,8.0]]  
    ])  

b=tf.reshape(a,[1,4,4,2])

max_pool_2x2 = tf.nn.max_pool(b,[1,2,2,1],[1,2,2,1],padding='VALID')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('池化前原始，image：\n')
    print(sess.run(a),'\n')
    print(sess.run(a).shape,'\n')
    print('池化前，reshape，image：\n')
    print(sess.run(b),'\n')
    print(sess.run(b).shape,'\n')
    print('池化后image：\n')
    print(sess.run(max_pool_2x2),'\n')
    print(sess.run(max_pool_2x2).shape,'\n')

池化前原始，image：

[[[1. 2. 3. 4.]
  [5. 6. 7. 8.]
  [8. 7. 6. 5.]
  [4. 3. 2. 1.]]

 [[4. 3. 2. 1.]
  [8. 7. 6. 5.]
  [1. 2. 3. 4.]
  [5. 6. 7. 8.]]] 

(2, 4, 4) 

池化前，reshape，image：

[[[[1. 2.]
   [3. 4.]
   [5. 6.]
   [7. 8.]]

  [[8. 7.]
   [6. 5.]
   [4. 3.]
   [2. 1.]]

  [[4. 3.]
   [2. 1.]
   [8. 7.]
   [6. 5.]]

  [[1. 2.]
   [3. 4.]
   [5. 6.]
   [7. 8.]]]] 

(1, 4, 4, 2) 

池化后image：

[[[[8. 7.]
   [7. 8.]]

  [[4. 4.]
   [8. 8.]]]] 

(1, 2, 2, 2) 



## tf.nn.dropout
[TF-tf.nn.dropout介绍](https://www.cnblogs.com/qggg/p/6849881.html)
````python
tf.nn.dropout(
            x, # 输入tensor
            keep_prob=None, #每个元素保留下来的概率
            # 一维int32张量，代表随机产生保留/丢弃表示的shape
            noise_shape=None, 
            seed=None, # 随机种子
            name=None, # name
            rate=None,
)
````

In [28]:
import tensorflow as tf
import numpy as np
x = tf.Variable(tf.ones([10,10]))
inputs = tf.nn.dropout(x, 0.8)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('原始数据：\n')
    print(sess.run(x),'\n')
    print(sess.run(x).shape,'\n')
    print('dropout：\n')
    print(sess.run(inputs),'\n')

原始数据：

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]] 

(10, 10) 

dropout：

[[1.25 1.25 0.   1.25 1.25 1.25 1.25 1.25 1.25 0.  ]
 [1.25 1.25 1.25 0.   1.25 1.25 1.25 1.25 1.25 1.25]
 [1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 0.  ]
 [1.25 1.25 1.25 1.25 1.25 1.25 1.25 0.   1.25 1.25]
 [1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25]
 [1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25]
 [1.25 0.   1.25 1.25 1.25 1.25 0.   0.   1.25 1.25]
 [1.25 0.   1.25 0.   1.25 1.25 1.25 1.25 0.   1.25]
 [1.25 1.25 1.25 1.25 0.   1.25 0.   1.25 0.   1.25]
 [1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25 1.25]] 



## tf.reshape
````python
tf.reshape(tensor, shape, name=None)
# shape里最多有一个维度的值可以填写为-1，表示自动计算此维度。
````
根据shape为[5,8]的tensor，生成一个新的tensor

In [29]:
alist = [[1, 2, 3, 4, 5, 6 ,7, 8],
         [7, 6 ,5 ,4 ,3 ,2, 1, 0],
         [3, 3, 3, 3, 3, 3, 3, 3],
         [1, 1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2, 2]]
oriarray = tf.constant(alist)
oplist = []
a1 = tf.reshape(oriarray, [1, 2, 5, 4])
oplist.append([a1, 'case 1, 2, 5, 4'])

a1 = tf.reshape(oriarray, [-1, 2, 5, 4])
oplist.append([a1, 'case -1, 2, 5, 4'])

a1 = tf.reshape(oriarray, [8, 5, 1, 1])
oplist.append([a1, 'case 8, 5, 1, 1'])

with tf.Session() as asess:
    for aop in oplist:
        print('--------{}---------'.format(aop[1]))
        print(asess.run(aop[0]))
        print('--------------------------\n\n')

--------case 1, 2, 5, 4---------
[[[[1 2 3 4]
   [5 6 7 8]
   [7 6 5 4]
   [3 2 1 0]
   [3 3 3 3]]

  [[3 3 3 3]
   [1 1 1 1]
   [1 1 1 1]
   [2 2 2 2]
   [2 2 2 2]]]]
--------------------------


--------case -1, 2, 5, 4---------
[[[[1 2 3 4]
   [5 6 7 8]
   [7 6 5 4]
   [3 2 1 0]
   [3 3 3 3]]

  [[3 3 3 3]
   [1 1 1 1]
   [1 1 1 1]
   [2 2 2 2]
   [2 2 2 2]]]]
--------------------------


--------case 8, 5, 1, 1---------
[[[[1]]

  [[2]]

  [[3]]

  [[4]]

  [[5]]]


 [[[6]]

  [[7]]

  [[8]]

  [[7]]

  [[6]]]


 [[[5]]

  [[4]]

  [[3]]

  [[2]]

  [[1]]]


 [[[0]]

  [[3]]

  [[3]]

  [[3]]

  [[3]]]


 [[[3]]

  [[3]]

  [[3]]

  [[3]]

  [[1]]]


 [[[1]]

  [[1]]

  [[1]]

  [[1]]

  [[1]]]


 [[[1]]

  [[1]]

  [[2]]

  [[2]]

  [[2]]]


 [[[2]]

  [[2]]

  [[2]]

  [[2]]

  [[2]]]]
--------------------------




## TF-variable_scope和name_scope区别
1、无论是variable_scope还是name_scope都对ops和tf.Variable生效。

2、只有tf.get_variable在name_scope下是不生效的。

3、这玩意真不好记，还是把这篇东西贴出来，以后忘记了过来看看。

In [30]:
with tf.variable_scope('variable_scope1'):
    with tf.variable_scope('variable_scope2'):
        v1 = tf.get_variable('v1', [1])
        v2 = tf.Variable(0,name='v2')
        x1 = 1.0 + v1
print('v1.name={},v1.op.name={}'.format(v1.name, v1.op.name))
print('v2.name={},v2.op.name={}'.format(v1.name, v1.op.name))
print('x1.name={},x1.op.name={}'.format(x1.name, x1.op.name))

print('--------------------------------------')

v1.name=variable_scope1/variable_scope2/v1:0,v1.op.name=variable_scope1/variable_scope2/v1
v2.name=variable_scope1/variable_scope2/v1:0,v2.op.name=variable_scope1/variable_scope2/v1
x1.name=variable_scope1/variable_scope2/add:0,x1.op.name=variable_scope1/variable_scope2/add
--------------------------------------


In [31]:
with tf.name_scope('name_scope1'):
    with tf.name_scope('name_scope2'):
        v11 = tf.get_variable('v11', [1])
        v12 = tf.Variable(0,name='v12')
        x11 = 1.0 + v11
print('v11.name={},v11.op.name={}'.format(v11.name, v11.op.name))
print('v12.name={},v12.op.name={}'.format(v12.name, v12.op.name))
print('x11.name={},x11.op.name={}'.format(x11.name, x11.op.name))

print('--------------------------------------')

v11.name=v11:0,v11.op.name=v11
v12.name=name_scope1/name_scope2/v12:0,v12.op.name=name_scope1/name_scope2/v12
x11.name=name_scope1/name_scope2/add:0,x11.op.name=name_scope1/name_scope2/add
--------------------------------------


In [32]:
with tf.variable_scope("variable_scope3"):
    with tf.name_scope("name_scope3"):
        v21 = tf.get_variable('v21', [1])
        v22 = tf.Variable(0,name='v22')
        x21 = 1.0 + v21
print('v21.name={},v21.op.name={}'.format(v21.name, v21.op.name))
print('v22.name={},v22.op.name={}'.format(v22.name, v22.op.name))
print('x21.name={},x21.op.name={}'.format(x21.name, x21.op.name))
print('--------------------------------------')

v21.name=variable_scope3/v21:0,v21.op.name=variable_scope3/v21
v22.name=variable_scope3/name_scope3/v22:0,v22.op.name=variable_scope3/name_scope3/v22
x21.name=variable_scope3/name_scope3/add:0,x21.op.name=variable_scope3/name_scope3/add
--------------------------------------


## CNN手写数字识别

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# x input  [batch,in_height,in_width,in_channels]
# [?,784] reshape -> [-1,28,28,1]
x_image = tf.reshape(x,[-1,28,28,1])

#w        [filter_height,filter_width,in_channels, out_channels]
W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 28*28*1 的图片卷积之后变为 -1*28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 -1*14*14*32
h_pool1 = max_pool_2x2(h_conv1)

# 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_vairable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)

# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2)


# 第一个全连接层
W_fc1 = weight_variable([7*7*64,1024]) #上一层有7*7*64个神经元，全连接层有1024个神经元
b_fc1 = bias_vairable([1024])#1024个节点

# 把池化层2的输出扁平化为1维向量
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_vairable([10])
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2

#计算输出
prediction = tf.nn.sigmoid(logits)

#交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
#AdamOptimizer
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

prediction_2 = tf.nn.softmax(prediction)
correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1)))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input

## 重构代码

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_size = 128
n_batch = mnist.train.num_examples // batch_size
max_step = 1000
keep_ = 0.8
log_dir = "Logs/log-6.1"

# 生成权重
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')

# 生成偏差
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape),name='b')

# 记录变量
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME',name='conv2d')
        
def conv_layer(input_tensor, weight_shape, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable(weight_shape)
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_vairable([weight_shape[-1]])
            variable_summaries(biases)
        with tf.name_scope('conv_comput'):
            preactivate = conv2d(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = act(preactivate)
        return activations

def linear_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_vairable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('linear_comput'):
            preactivate = tf.matmul(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = act(preactivate)
        return activations
        

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Max_pool')

with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,784],name='input_x')
    with tf.name_scope('Input_reshape'):
        x_image = tf.reshape(x,[-1,28,28,1],name='x-image')
        tf.summary.image('input',x_image,10)
    y = tf.placeholder(tf.float32,[None,10],name='input_y')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')

# 第一次卷积   28*28*1->28*28*32
conv_layer1 = conv_layer(x_image,[5,5,1,32],'conv_layer1')
# 池化之后变为 14*14*32
with tf.name_scope('Max_pool1'):
    h_pool1 = max_pool_2x2(conv_layer1)

# 第二次卷积 14*14*32->14*14*64
conv_layer2 = conv_layer(h_pool1,[5,5,32,64],'conv_layer2')
# 第二次池化之后变为 7*7*64
with tf.name_scope('Max_pool2'):
    h_pool2 = max_pool_2x2(conv_layer2)

with tf.name_scope('Flatten'):
    flatten_ = tf.reshape(h_pool2,[-1,7*7*64])
    
# 第一个全连接层 7*7*64 - 1024
fc1 = linear_layer(flatten_, 7*7*64, 1024, 'FC1')

with tf.name_scope('Dropput'):
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
# 第二个全连接层 1024 - 10
logits = linear_layer(fc1_drop, 1024, 10, 'FC2',act=tf.nn.sigmoid)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.name_scope('accuracy'):
    prediction = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
merged = tf.summary.merge_all()

def get_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(batch_size)
        k = keep_
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x:xs, y:ys, keep_prob: k}

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(max_step):
        if i%10 == 0:
            summary,acc = sess.run([merged,accuracy], feed_dict=get_dict(False))
            test_writer.add_summary(summary, i)
            print("Step: " + str(i) + ", acc: " + str(acc))
        else:
            summary,_ = sess.run([merged,train_step], feed_dict=get_dict(True))
            train_writer.add_summary(summary,i)
        
    train_writer.close()
    test_writer.close()

Step: 0, acc: 0.11
Step: 10, acc: 0.138
Step: 20, acc: 0.1994
Step: 30, acc: 0.3193
Step: 40, acc: 0.2373
Step: 50, acc: 0.2421
Step: 60, acc: 0.527
Step: 70, acc: 0.7946
Step: 80, acc: 0.7877
Step: 90, acc: 0.8439
Step: 100, acc: 0.8235
Step: 110, acc: 0.8555
Step: 120, acc: 0.8639
Step: 130, acc: 0.8655
Step: 140, acc: 0.8672
Step: 150, acc: 0.8711
Step: 160, acc: 0.8726
Step: 170, acc: 0.8754
Step: 180, acc: 0.8746
Step: 190, acc: 0.8771
Step: 200, acc: 0.8782
Step: 210, acc: 0.8805
Step: 220, acc: 0.8806
Step: 230, acc: 0.8763
Step: 240, acc: 0.8815
Step: 250, acc: 0.8805
Step: 260, acc: 0.877
Step: 270, acc: 0.8811
Step: 280, acc: 0.8824
Step: 290, acc: 0.8827
Step: 300, acc: 0.8829
Step: 310, acc: 0.8843
Step: 320, acc: 0.8853
Step: 330, acc: 0.8838
Step: 340, acc: 0.8837
Step: 350, acc: 0.8833
Step: 360, acc: 0.8853
Step: 370, acc: 0.885
Step: 380, acc: 0.8841
Step: 390, acc: 0.8871
Step: 400, acc: 0.8871
Step: 410, acc: 0.8877
Step: 420, acc: 0.8874
Step: 430, acc: 0.8888
Step: